## Install lib

In [1]:
%%bash
rm -rf /usr/local/cuda
ln -s /usr/local/cuda-10.1 /usr/local/cuda

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [3]:
!stat /usr/local/cuda

  File: /usr/local/cuda -> /usr/local/cuda-10.1
  Size: 20        	Blocks: 0          IO Block: 4096   symbolic link
Device: 34h/52d	Inode: 3028158     Links: 1
Access: (0777/lrwxrwxrwx)  Uid: (    0/    root)   Gid: (    0/    root)
Access: 2021-11-22 11:48:01.494879953 +0000
Modify: 2021-11-22 11:48:01.447879968 +0000
Change: 2021-11-22 11:48:01.447879968 +0000
 Birth: -


In [4]:
!pip uninstall triton

In [ ]:
!pip uninstall torch

In [5]:
%%bash
export LD_LIBRARY_PATH=/usr/lib/

In [7]:
!apt-get install clang-9 llvm-9 llvm-9-dev llvm-9-tools

Reading package lists... Done
Building dependency tree       
Reading state information... Done
clang-9 is already the newest version (1:9-2~ubuntu18.04.2).
llvm-9 is already the newest version (1:9-2~ubuntu18.04.2).
llvm-9-dev is already the newest version (1:9-2~ubuntu18.04.2).
llvm-9-tools is already the newest version (1:9-2~ubuntu18.04.2).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [ ]:
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

In [9]:
%%writefile setup.sh

git clone https://github.com/NVIDIA/apex
cd apex
pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

Writing setup.sh


In [ ]:
!sh setup.sh

In [ ]:
!pip install triton==0.2.3

In [9]:
# !pip uninstall -y typing

In [ ]:
!pip install cpufeature

In [ ]:
!DS_BUILD_CPU_ADAM=1 DS_BUILD_SPARSE_ATTN=1 pip install deepspeed==0.3.7

In [14]:
!ds_report

--------------------------------------------------
DeepSpeed C++/CUDA extension op report
--------------------------------------------------
NOTE: Ops not installed will be just-in-time (JIT) compiled at
      runtime if needed. Op compatibility means that your system
      meet the required dependencies to JIT install the op.
--------------------------------------------------
JIT compiled ops requires ninja
ninja .................. [OKAY]
--------------------------------------------------
op name ................ installed .. compatible
--------------------------------------------------
cpu_adam ............... [YES] ...... [OKAY]
fused_adam ............. [NO] ....... [OKAY]
fused_lamb ............. [NO] ....... [OKAY]
sparse_attn ............ [YES] ...... [OKAY]
transformer ............ [NO] ....... [OKAY]
stochastic_transformer . [NO] ....... [OKAY]
utils .................. [NO] ....... [OKAY]
--------------------------------------------------
DeepSpeed general environment info:
tor

In [15]:
import deepspeed.ops.sparse_attention.sparse_attn_op

In [1]:
!rm -rf ru-gpts

In [2]:
!git clone https://github.com/lauberto/rugpt3xl_ilm.git
 # TODO: add path to requirements.txt and install requirements (make sure to use the requirements necessary both for ILM and RUGP3XL)

Cloning into 'ru-gpts'...
remote: Enumerating objects: 673, done.
remote: Counting objects: 100% (168/168), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 673 (delta 103), reused 135 (delta 80), pack-reused 505
Receiving objects: 100% (673/673), 402.50 KiB | 3.73 MiB/s, done.
Resolving deltas: 100% (403/403), done.


In [ ]:
!pip install transformers==3.5.1

In [18]:
!cp rugpt3xl_ilm/src_utils/trainer_pt_utils.py /usr/local/lib/python3.7/dist-packages/transformers/trainer_pt_utils.py

# Test model

### Load model

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import sys
sys.path.append("ru-gpts/")

In [5]:
import os
os.environ["USE_DEEPSPEED"] = "1"

## ILM

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd gdrive/My Drive/RUGPT3XL

In [ ]:
# TODO: upload data to gdrive
# TODO: use the right path for the script below
!! python train_ilm.py \
	experiment \
  sberbank_small \
	./data/Economics/ \
	--seed 0 \
	--train_examples_tag train \
  --data_loader_num_workers 8 \
  --eval_examples_tag valid \
  --eval_max_num_examples 512 \
  --tokenizer_name CUSTOM \
  --tokenizer_custom_vocab_fp rus_gpt2/vocab.json \
  --model_name sberbank-ai/rugpt3small_based_on_gpt2 2>&1 | tee sberbank_CAT_TrainerLog.txt